# Attention readout for examples

In [1]:
import pandas as pd
from attention_readout import gather_attention, gather_attention_model, to_markdown

/home/jgut/miniconda3/envs/SMILES_OR_SELFIES/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/jgut/miniconda3/envs/SMILES_OR_SELFIES/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [4]:
gather_attention("CC=CCOCC")

'## Molecule\\ CC=CCOCC\\ ## SMILES atomwise\\ |    |      C |     C |      = |     C |     C |      O |     C |      C |\n|---:|-------:|------:|-------:|------:|------:|-------:|------:|-------:|\n|  0 | -0.056 | 0.029 | -0.023 | 0.038 | 0.005 | -0.006 | 0.029 | -0.016 |\n|  0 |  0.069 | 0.154 |  0.102 | 0.163 | 0.13  |  0.119 | 0.154 |  0.109 |\\ ## SELFIES atomwise\\ |    |    [C] |   [C] |   [=C] |    [C] |    [O] |    [C] |    [C] |\n|---:|-------:|------:|-------:|-------:|-------:|-------:|-------:|\n|  0 | -0.039 | 0.169 |  0.029 | -0.041 | -0.046 | -0.041 | -0.032 |\n|  0 |  0.104 | 0.312 |  0.172 |  0.102 |  0.097 |  0.102 |  0.111 |\\ ## SMILES SentencePiece\\ |    |   CC=CC |   OCC |\n|---:|--------:|------:|\n|  0 |  -0.358 | 0.358 |\n|  0 |   0.142 | 0.858 |\\ ## SELFIES SentencePiece\\ |    |   [C][C][=C][ |   C][O][C][ |    C] |\n|---:|--------------:|------------:|------:|\n|  0 |        -0.253 |      -0.202 | 0.456 |\n|  0 |         0.08  |       0.131 | 0.789 |\\ '